In [1]:
import sqlalchemy as db
import pandas as pd
import json
import os
from datetime import datetime, timezone

import copy

# Time execution
from time import time

## Functions

In [2]:
def format_playlist(playlist):
    # Change the format of important columns in the playlist
    # Also remove the track_list and return it to process it later
    
    # Store the tracks for later
    track_list = playlist.pop('tracks')
    
    # Convert modified_at column into Date object
    timestamp = playlist['modified_at']
    playlist['modified_at'] = datetime.fromtimestamp(timestamp, timezone.utc).date()
    
    # Convert collaborative column into boolean
    # I don't know what values they're using to represent true/false values.
    # The following code stops the creation of the database if we find a value
    # we don't recognize
    colab = str(playlist['collaborative'])
    if colab in true_values:
        playlist['collaborative'] = True
    elif colab in false_values:
        playlist['collaborative'] = False
    else:
        raise ValueError('The following value is not in the allowed true/false list:' +
                         str(playlist['collaborative']),
                         playlist['collaborative'])
    
    # Remove the pos entry from each track and return it in the same order
    pos_list = []
    for track in track_list:
        pos = track.pop('pos')
        pos_list.append(pos)
    
    return playlist, track_list, pos_list

def find_new_tracks(track_list, track_cache):
    # Remove duplicate tracks
    set_uniques = {tuple(d.items()) for d in track_list}
    track_unique = [dict(t) for t in set_uniques]
    
    # Check which tracks are cached already
    uris = [track['track_uri'] for track in track_unique]
    duplicate = [ (U in track_cache.keys()) for U in uris]
    
    # Retain only new tracks
    new_tracks = []
    count = len(track_cache)
    for i in range(len(track_unique)):
        if not duplicate[i]:
            new_tracks.append(track_unique[i])
            
            # Update cache
            track_cache[uris[i]] = count
            count += 1
    
    return new_tracks, track_cache

def create_pairings(track_list, pos_list, pid_list, track_cache):
    # Create pairings table
    pairing_list = []
    
    for i in range(len(track_list)):
        track = track_list[i]
        pid = pid_list[i]
        pos = pos_list[i]
        
        # tid is the index of the track in the cache
        tid = track_cache[track['track_uri']]
        
        # Create the playlist-track pairing
        pair = {
            'pid': pid,
            'tid': tid,
            'pos': pos
        }
        
        # Add the pairing to the list
        pairing_list.append(pair)
    
    return(pairing_list)

## Preview data

In [3]:
data_folder = "spotify_million_playlist_dataset/data"

# Load one json file and display the column names
json_files = [filename for filename in os.listdir(data_folder) if filename.endswith(".json")]
json_files.sort()

json_file = json_files[0]
json_file_path = os.path.join(data_folder, json_file)
with open(json_file_path) as json_file:
    data = json.load(json_file)

playlist = data['playlists'][0]

print(' Keys for playlists')
print('--------------------')
for key in data['playlists'][0].keys():
    print('   ' + key)
print()

print(' Keys for tracks')
print('-----------------')
for key in data['playlists'][0]['tracks'][0].keys():
    print('   ' + key)

 Keys for playlists
--------------------
   name
   collaborative
   pid
   modified_at
   num_tracks
   num_albums
   num_followers
   tracks
   num_edits
   duration_ms
   num_artists

 Keys for tracks
-----------------
   pos
   artist_name
   track_uri
   artist_uri
   track_name
   album_uri
   duration_ms
   album_name


## Create databases and connection

In [4]:
data_folder = "spotify_million_playlist_dataset/data"

engine = db.create_engine("sqlite:///spotify_mpd_test.db")
conn = engine.connect()
metadata = db.MetaData()

# Playlists table
tb_playlists = db.Table('playlists', metadata,
                        db.Column('name', db.String(255), nullable=False),
                        # db.Column('pid', db.Integer(), nullable=False),
                        db.Column('pid', db.Integer(), nullable=False, primary_key=True),
                        db.Column('collaborative', db.Boolean()),
                        db.Column('modified_at', db.Date()),
                        db.Column('num_tracks', db.Integer()),
                        db.Column('num_albums', db.Integer()),
                        db.Column('num_followers', db.Integer()),
                        db.Column('num_edits', db.Integer()),
                        db.Column('duration_ms', db.Integer()),
                        db.Column('num_artists', db.Integer())
                       )

# Tracks table
tb_tracks = db.Table('tracks', metadata,
                     db.Column('tid', db.Integer(), nullable=False, primary_key=True),
                     db.Column('track_name', db.String(255), nullable=False),
                     db.Column('artist_name', db.String(255)),
                     db.Column('album_name', db.String(255)),
                     db.Column('duration_ms', db.Integer()),
                     db.Column('track_uri', db.String(255), nullable=False),
                     db.Column('artist_uri', db.String(255)),
                     db.Column('album_uri', db.String(255))
                    )

# Table with playlist-track pairing
tb_pairings = db.Table('pairings', metadata,
                       db.Column('pid', db.Integer(), nullable=False),
                       db.Column('tid', db.Integer(), nullable=False),
                       db.Column('pos', db.Integer(), nullable=False)
                      )

metadata.create_all(engine) # Create the tables

true_values = ['true', '1']
false_values = ['false', '0']

Populate database. We iterate over all json files and insert all the playlists in a single file at the same time.

In [5]:
time_0 = time()    # Record starting time

# List to record all the tracks that are already in the database
# This prevents adding duplicate entries
track_cache = {}

# Load .json file names from disk
json_files = [filename for filename in os.listdir(data_folder) if filename.endswith(".json")]
json_files.sort()

# Limit to the first few files
json_files = json_files

# Iterate through the selected JSON files
for json_file_name in json_files:
    start_time = time()
    print('Processing:', json_file_name)
    
    # Load file
    json_file_path = os.path.join(data_folder, json_file_name)
    json_file = open(json_file_path)
    data = json.load(json_file)
    
    # Lists to store pairings
    new_track_pairing = []
    new_pos_list = []
    pid_list = []
    
    # Process the playlists
    for playlist in data["playlists"]:
        # Format the playlist info and obtain its tracks
        playlist, track_list, pos_list = format_playlist(playlist)
        
        # Save formated tracks and positions
        new_track_pairing = new_track_pairing + track_list
        new_pos_list = new_pos_list + pos_list
        pid_list = pid_list + [playlist['pid']]*len(track_list)
    
    # Remove duplicate tracks and update cache
    unseen_tracks, track_cache = find_new_tracks(new_track_pairing, track_cache)
    
    # Create the playlist-track pairings
    new_pairings = create_pairings(new_track_pairing, new_pos_list, pid_list, track_cache)
    
    # Insert playlists
    query = db.insert(tb_playlists)
    ResultProxy = conn.execute(query, data["playlists"])

    # Insert tracks
    query = db.insert(tb_tracks)
    ResultProxy = conn.execute(query, unseen_tracks)

    # Insert pairings
    query = db.insert(tb_pairings)
    ResultProxy = conn.execute(query, new_pairings)
    
    current_time = time()
    print('Success!')
    print('%0.3f sec' % (current_time-start_time) )
    print()

# Commit changes to the database
# conn.commit()

current_time = time()
print('END')
print('Total time: %0.3f sec' % float(current_time-time_0) )
print('Total time: %0.3f min' % float((current_time-time_0)/60) )

Processing: mpd.slice.0-999.json
Success!
1.304 sec

Processing: mpd.slice.1000-1999.json
Success!
1.029 sec

Processing: mpd.slice.10000-10999.json
Success!
1.068 sec

Processing: mpd.slice.100000-100999.json
Success!
1.106 sec

Processing: mpd.slice.101000-101999.json
Success!
0.982 sec

Processing: mpd.slice.102000-102999.json
Success!
1.091 sec

Processing: mpd.slice.103000-103999.json
Success!
0.996 sec

Processing: mpd.slice.104000-104999.json
Success!
1.119 sec

Processing: mpd.slice.105000-105999.json
Success!
0.995 sec

Processing: mpd.slice.106000-106999.json
Success!
1.038 sec

Processing: mpd.slice.107000-107999.json
Success!
0.949 sec

Processing: mpd.slice.108000-108999.json
Success!
1.054 sec

Processing: mpd.slice.109000-109999.json
Success!
0.986 sec

Processing: mpd.slice.11000-11999.json
Success!
0.948 sec

Processing: mpd.slice.110000-110999.json
Success!
0.936 sec

Processing: mpd.slice.111000-111999.json
Success!
0.986 sec

Processing: mpd.slice.112000-112999.json

Success!
1.059 sec

Processing: mpd.slice.220000-220999.json
Success!
0.941 sec

Processing: mpd.slice.221000-221999.json
Success!
0.925 sec

Processing: mpd.slice.222000-222999.json
Success!
1.038 sec

Processing: mpd.slice.223000-223999.json
Success!
0.962 sec

Processing: mpd.slice.224000-224999.json
Success!
0.941 sec

Processing: mpd.slice.225000-225999.json
Success!
0.996 sec

Processing: mpd.slice.226000-226999.json
Success!
0.898 sec

Processing: mpd.slice.227000-227999.json
Success!
0.987 sec

Processing: mpd.slice.228000-228999.json
Success!
0.972 sec

Processing: mpd.slice.229000-229999.json
Success!
0.975 sec

Processing: mpd.slice.23000-23999.json
Success!
0.915 sec

Processing: mpd.slice.230000-230999.json
Success!
0.935 sec

Processing: mpd.slice.231000-231999.json
Success!
0.880 sec

Processing: mpd.slice.232000-232999.json
Success!
0.920 sec

Processing: mpd.slice.233000-233999.json
Success!
0.972 sec

Processing: mpd.slice.234000-234999.json
Success!
1.064 sec

Proces

Success!
0.869 sec

Processing: mpd.slice.342000-342999.json
Success!
0.932 sec

Processing: mpd.slice.343000-343999.json
Success!
0.965 sec

Processing: mpd.slice.344000-344999.json
Success!
0.891 sec

Processing: mpd.slice.345000-345999.json
Success!
0.900 sec

Processing: mpd.slice.346000-346999.json
Success!
0.984 sec

Processing: mpd.slice.347000-347999.json
Success!
0.989 sec

Processing: mpd.slice.348000-348999.json
Success!
0.967 sec

Processing: mpd.slice.349000-349999.json
Success!
0.957 sec

Processing: mpd.slice.35000-35999.json
Success!
0.956 sec

Processing: mpd.slice.350000-350999.json
Success!
0.967 sec

Processing: mpd.slice.351000-351999.json
Success!
0.922 sec

Processing: mpd.slice.352000-352999.json
Success!
0.893 sec

Processing: mpd.slice.353000-353999.json
Success!
0.972 sec

Processing: mpd.slice.354000-354999.json
Success!
0.936 sec

Processing: mpd.slice.355000-355999.json
Success!
0.927 sec

Processing: mpd.slice.356000-356999.json
Success!
0.983 sec

Proces

Success!
0.999 sec

Processing: mpd.slice.464000-464999.json
Success!
1.033 sec

Processing: mpd.slice.465000-465999.json
Success!
0.952 sec

Processing: mpd.slice.466000-466999.json
Success!
1.016 sec

Processing: mpd.slice.467000-467999.json
Success!
0.937 sec

Processing: mpd.slice.468000-468999.json
Success!
1.026 sec

Processing: mpd.slice.469000-469999.json
Success!
0.935 sec

Processing: mpd.slice.47000-47999.json
Success!
0.995 sec

Processing: mpd.slice.470000-470999.json
Success!
0.976 sec

Processing: mpd.slice.471000-471999.json
Success!
0.957 sec

Processing: mpd.slice.472000-472999.json
Success!
0.985 sec

Processing: mpd.slice.473000-473999.json
Success!
1.002 sec

Processing: mpd.slice.474000-474999.json
Success!
0.914 sec

Processing: mpd.slice.475000-475999.json
Success!
1.012 sec

Processing: mpd.slice.476000-476999.json
Success!
0.945 sec

Processing: mpd.slice.477000-477999.json
Success!
0.962 sec

Processing: mpd.slice.478000-478999.json
Success!
1.012 sec

Proces

Success!
0.977 sec

Processing: mpd.slice.586000-586999.json
Success!
1.003 sec

Processing: mpd.slice.587000-587999.json
Success!
0.937 sec

Processing: mpd.slice.588000-588999.json
Success!
0.945 sec

Processing: mpd.slice.589000-589999.json
Success!
0.935 sec

Processing: mpd.slice.59000-59999.json
Success!
0.994 sec

Processing: mpd.slice.590000-590999.json
Success!
0.917 sec

Processing: mpd.slice.591000-591999.json
Success!
1.014 sec

Processing: mpd.slice.592000-592999.json
Success!
0.969 sec

Processing: mpd.slice.593000-593999.json
Success!
0.953 sec

Processing: mpd.slice.594000-594999.json
Success!
0.977 sec

Processing: mpd.slice.595000-595999.json
Success!
1.000 sec

Processing: mpd.slice.596000-596999.json
Success!
0.899 sec

Processing: mpd.slice.597000-597999.json
Success!
1.034 sec

Processing: mpd.slice.598000-598999.json
Success!
1.012 sec

Processing: mpd.slice.599000-599999.json
Success!
0.891 sec

Processing: mpd.slice.6000-6999.json
Success!
0.945 sec

Processing

Success!
0.978 sec

Processing: mpd.slice.707000-707999.json
Success!
0.995 sec

Processing: mpd.slice.708000-708999.json
Success!
0.946 sec

Processing: mpd.slice.709000-709999.json
Success!
0.922 sec

Processing: mpd.slice.71000-71999.json
Success!
0.950 sec

Processing: mpd.slice.710000-710999.json
Success!
0.924 sec

Processing: mpd.slice.711000-711999.json
Success!
0.996 sec

Processing: mpd.slice.712000-712999.json
Success!
0.963 sec

Processing: mpd.slice.713000-713999.json
Success!
0.968 sec

Processing: mpd.slice.714000-714999.json
Success!
0.905 sec

Processing: mpd.slice.715000-715999.json
Success!
0.929 sec

Processing: mpd.slice.716000-716999.json
Success!
1.048 sec

Processing: mpd.slice.717000-717999.json
Success!
0.990 sec

Processing: mpd.slice.718000-718999.json
Success!
0.956 sec

Processing: mpd.slice.719000-719999.json
Success!
0.853 sec

Processing: mpd.slice.72000-72999.json
Success!
1.038 sec

Processing: mpd.slice.720000-720999.json
Success!
0.967 sec

Processi

Success!
0.934 sec

Processing: mpd.slice.829000-829999.json
Success!
1.037 sec

Processing: mpd.slice.83000-83999.json
Success!
0.914 sec

Processing: mpd.slice.830000-830999.json
Success!
1.002 sec

Processing: mpd.slice.831000-831999.json
Success!
0.994 sec

Processing: mpd.slice.832000-832999.json
Success!
0.933 sec

Processing: mpd.slice.833000-833999.json
Success!
1.003 sec

Processing: mpd.slice.834000-834999.json
Success!
1.058 sec

Processing: mpd.slice.835000-835999.json
Success!
0.948 sec

Processing: mpd.slice.836000-836999.json
Success!
0.938 sec

Processing: mpd.slice.837000-837999.json
Success!
0.993 sec

Processing: mpd.slice.838000-838999.json
Success!
0.994 sec

Processing: mpd.slice.839000-839999.json
Success!
1.004 sec

Processing: mpd.slice.84000-84999.json
Success!
0.971 sec

Processing: mpd.slice.840000-840999.json
Success!
0.962 sec

Processing: mpd.slice.841000-841999.json
Success!
0.957 sec

Processing: mpd.slice.842000-842999.json
Success!
0.959 sec

Processi

Success!
0.967 sec

Processing: mpd.slice.950000-950999.json
Success!
1.032 sec

Processing: mpd.slice.951000-951999.json
Success!
0.951 sec

Processing: mpd.slice.952000-952999.json
Success!
0.905 sec

Processing: mpd.slice.953000-953999.json
Success!
0.977 sec

Processing: mpd.slice.954000-954999.json
Success!
0.955 sec

Processing: mpd.slice.955000-955999.json
Success!
1.014 sec

Processing: mpd.slice.956000-956999.json
Success!
0.974 sec

Processing: mpd.slice.957000-957999.json
Success!
0.887 sec

Processing: mpd.slice.958000-958999.json
Success!
0.897 sec

Processing: mpd.slice.959000-959999.json
Success!
1.000 sec

Processing: mpd.slice.96000-96999.json
Success!
0.907 sec

Processing: mpd.slice.960000-960999.json
Success!
0.859 sec

Processing: mpd.slice.961000-961999.json
Success!
0.908 sec

Processing: mpd.slice.962000-962999.json
Success!
0.928 sec

Processing: mpd.slice.963000-963999.json
Success!
0.870 sec

Processing: mpd.slice.964000-964999.json
Success!
0.983 sec

Proces

In [ ]:
# Commit changes to the database
trans = conn.begin()
try:
    trans.commit()
except:
    trans.rollback()
    raise
finally:
    trans.close()


In [7]:
# Find number of pairings
time_0 = time()    # Record starting time

# Load .json file names from disk
json_files = [filename for filename in os.listdir(data_folder) if filename.endswith(".json")]
json_files.sort()

# Iterate through the selected JSON files
num_pairings = 0
for json_file_name in json_files:
    # Load file
    json_file_path = os.path.join(data_folder, json_file_name)
    json_file = open(json_file_path)
    data = json.load(json_file)
    
    # Process the playlists
    for playlist in data["playlists"]:
        num_tracks = len(playlist['tracks'])
        num_pairings += num_tracks

current_time = time()
print('END')
print('Total time: %0.3f sec' % float(current_time-time_0) )
print('Total time: %0.3f min' % float((current_time-time_0)/60) )

END
Total time: 149.278 sec
Total time: 2.488 min


In [8]:
print('Number of pairings:', num_pairings)

Number of pairings: 66346428


## Display the resulting databases

In [ ]:
engine = db.create_engine("sqlite:///spotify_mpd_test.db")
conn = engine.connect()

In [ ]:
pd.read_sql_query('SELECT * FROM playlists', conn).sort_values('pid')

In [ ]:
pd.read_sql_query('SELECT * FROM tracks', conn)

In [ ]:
pd.read_sql_query('SELECT * FROM pairings', conn)

In [6]:
results = conn.execute(db.text("SELECT COUNT(*) FROM pairings"))

N = results.fetchall()[0][0]
print(N)

66346428


In [9]:
# close the database connection
conn.close()
engine.dispose()

In [ ]:
playlists = pd.read_sql_query('SELECT * FROM playlists', conn)
playlists = playlists.sort_values('pid')

pid = playlists['pid']
pid